In [19]:
%load_ext autoreload
%autoreload 2

In [20]:
from food.qdrant import *

In [16]:
from food.paths import *
import pandas as pd
from food.psql import *
from food.tools import *

In [17]:
data_path.ls()

[PosixPath('/home/dima/data/reserence_images'),
 PosixPath('/home/dima/data/full'),
 PosixPath('/home/dima/data/app1.log'),
 PosixPath('/home/dima/data/branded'),
 PosixPath('/home/dima/data/FoundationFoods'),
 PosixPath('/home/dima/data/FNDDS'),
 PosixPath('/home/dima/data/support_data')]

In [18]:
ff = (data_path/'full')

In [24]:
df = pd.read_csv(ff/'food.csv')

In [47]:
df = df[df['data_type']=='foundation_food']

In [61]:
df = df[['description']].reset_index()
df = df.rename(columns = {'index':'id'})

In [67]:
df

,id,description
0,0,"Hummus, commercial"
1,1,"Milk, reduced fat, fluid, 2% milkfat, with add..."
2,2,"Tomatoes, grape, raw"
3,3,"Salt, table, iodized"
4,4,"Beans, snap, green, canned, regular pack, drai..."
...,...,...
218,218,"Mushroom, enoki"
219,219,"Mushroom, crimini"
220,220,"Mushroom, maitake"
221,221,"Mushroom, beech"


In [68]:
df.to_sql('fundation_foods',schema='food',if_exists='append',con=engine,index=False)

In [42]:
from food.qdrant import *

In [71]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm
import food.custom_pandas as cpd

from food.qdrant import *


project_name = "food"
dim = 768
limit = 100000



In [90]:
table = 'fundation_foods'

In [91]:
project_name

'food'

In [92]:
df

,id,description,clip
0,0,"Hummus, commercial","[-0.02546718, -0.013269787, -0.009073956, -0.0..."
1,1,"Milk, reduced fat, fluid, 2% milkfat, with add...","[0.0063349595, 0.013251174, 0.006614223, 0.056..."
2,2,"Tomatoes, grape, raw","[0.06749406, 0.011973576, 0.0036271457, -0.016..."
3,3,"Salt, table, iodized","[0.0014464495, 0.04146881, 0.0153899435, -0.02..."
4,4,"Beans, snap, green, canned, regular pack, drai...","[0.030651499, 0.045894135, 0.029624652, 0.0097..."
...,...,...,...
218,218,"Mushroom, enoki","[0.06986512, 0.023284465, 0.02792236, -0.00119..."
219,219,"Mushroom, crimini","[0.059809633, 0.026994426, 0.022661524, 0.0117..."
220,220,"Mushroom, maitake","[0.051199224, 0.0240749, 0.023576245, 0.008109..."
221,221,"Mushroom, beech","[0.05649968, 0.06563859, 0.018384956, 0.004621..."


In [95]:


query = f"""select f.id,f.description, f.clip as clip
        FROM      {project_name}.{table}    f

        WHERE clip    is not null
        """



for df in tqdm(cpd.read_sql_query(query, engine, chunksize=limit), desc="qdrant_update {project_name}"):

    ids = df['id'].tolist()
    clip = df['clip']
    clip = np.array([np.array(c) for c in clip.values])

    df = df.fillna('0')


    payload = df.drop(columns = ['id','clip']).to_dict('records')

    client.upload_collection(
        collection_name='fundation_food',
        vectors=clip,
        payload=payload,
        ids=ids,
        parallel=4
    )

qdrant_update {project_name}: 0it [00:00, ?it/s]2022-05-16 00:37:37.825 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 0 PID: 121242
2022-05-16 00:37:37.844 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 1 PID: 121245
2022-05-16 00:37:37.860 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 2 PID: 121248
2022-05-16 00:37:37.875 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 3 PID: 121251

2022-05-16 00:37:37.967 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 3 finished2022-05-16 00:37:38.034 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 1 finished
2022-05-16 00:37:38.075 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 0 finished
4it [00:00, 14.61it/s]
qdrant_update {project_name}: 1it [00:00,  3.06it/s]2022-05-16 00:37:38.080 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 2 finish

In [96]:
client.collection_len('fundation_food')

223